# Kaggle Quora Question pair 

## Load model and data set
- We will use the dataset from Kaggle Competition and Glove, read the README file for more information
- Import important model. For this problem, we will use Logistic Regression, Naive Bayes and XGBoost algorithm 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk

train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

# Any results you write to the current directory are saved as output.

C:\Users\minhq\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
import random
import re

C:\Users\minhq\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
#Evaluating metrics
def precision_recall (predicted, label):
    predicted = list(predicted)
    label = list(label)
    true_positive = 0
    false_negative = 0
    false_positive = 0
    for i in range (len(label)):
        if (label[i] == 1 and predicted[i] == 1):
            true_positive +=1
        if (label[i] == 0 and predicted[i] == 0):
            false_positive +=1
        if (label[i] == 0 and predicted[i] == 1):
            false_negative +=1
    print(true_positive)
    print(false_positive)
    print(false_negative)
    precision = true_positive /(true_positive + false_positive)
    recall = true_positive /(true_positive + false_negative)
    return [precision, recall]

In [5]:
train = train.fillna('empty')
test = test.fillna('empty')
len(train)

404290

## Word Vectorization
- For this problem, we use Glove word pre-train word vectorization. 
- For each question, we sum up all the vector of each word and average them to get the vectorization of a question

In [16]:
def createWordVector(datafile):
    glove = open("./data/{}".format(datafile), encoding="utf8")
    glove_dict = {}
    for line in glove:
        line = line.split(" ")
        glove_dict[line.pop(0)] = line
    return glove_dict

In [17]:
glove50d_dict = createWordVector("glove.6B.50d.txt")
glove50d_df = pd.DataFrame(glove50d_dict)

# glove100d_dict = createWordVector('glove.6B.100d.txt')
# glove100d_df = pd.DataFrame (glove100d_dict)

# glove200d_dict = createWordVector("glove.6B.200d.txt")
# glove200d_df = pd.DataFrame (glove200d_dict)

In [18]:
#function to create a random vector, this is because we want to generate random vector for unknown words
def create_ran_array (d):
    arr = []
    for i in range (d):
        arr.append(random.uniform(0,1))
    return arr

#convert each question to vector 
def convertToV (question, glove_dict = glove50d_dict, d = 50):
    x = np.array([0]*d, dtype = np.float64)
    question = nltk.word_tokenize (question)
    count = 0
    for word in question:
        if word not in glove_dict:
            glove_dict[word] = create_ran_array (d)
            x1 = np.array(glove_dict[word], dtype = np.float64)
        else:
            x1 = np.array(glove_dict[word], dtype = np.float64)
        x = np.add(x, x1)
        count +=1
    return x/count

In [19]:
small_train = train.sample(5000)
small_test = test.sample(5000)
print(len(train))

404290


In [20]:
# apply the convertToV function to every question in the database
X_question1_train = small_train.question1.apply(convertToV)
X_question2_train = small_train.question2.apply(convertToV)

X_question1_test = small_test.question1.apply(convertToV)
X_question2_test = small_test.question2.apply(convertToV)

## Logistic Regression

In [21]:
X = list(abs (X_question1_train - X_question2_train))
X = np.nan_to_num(X)
X_test = list(abs(X_question1_test - X_question2_test))    
y = small_train['is_duplicate'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [23]:
pred_lr = lr.predict_proba(X_test)
pred = lr.predict(X_test)
logloss_lr = log_loss(y_test, pred_lr)
print(logloss_lr)
print(np.mean (pred == y_test))
print(precision_recall(pred, y_test))

0.5996975737827522
0.656
123
533
88
[0.1875, 0.5829383886255924]


In [24]:
small_train.questions= small_train.question1 + small_train.question2

tfidf = TfidfVectorizer(analyzer = 'word',
                        stop_words = 'english',
                        lowercase = True,
                        max_features = 300,
                        norm = 'l1')

BagOfWords = pd.concat([small_train.question1, small_train.question2], axis = 0)
tfidf.fit(BagOfWords)

train_q1_tfidf = tfidf.transform(small_train.question1)
train_q2_tfidf = tfidf.transform(small_train.question2)

X = abs(train_q1_tfidf - train_q2_tfidf)
y = small_train['is_duplicate'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

C:\Users\minhq\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [25]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [26]:
pred_lr = lr.predict_proba(X_test)
logloss_lr = log_loss(y_test, pred_lr)
print(logloss_lr)

0.6323834700140135


In [28]:
# nltk.download('stopwords')
stops = set(stopwords.words("english"))
def word_match_share(row):
    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().split():
        if word not in stops:
            q1words[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    shared_words_in_q1 = [w for w in q1words.keys() if w in q2words]
    shared_words_in_q2 = [w for w in q2words.keys() if w in q1words]
    R = (len(shared_words_in_q1) + len(shared_words_in_q2))/(len(q1words) + len(q2words))
    return R

def tfidf_word_match_share(row):
    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().split():
        if word not in stops:
            q1words[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    
    shared_weights = [weights.get(w, 0) for w in q1words.keys() if w in q2words] + [weights.get(w, 0) for w in q2words.keys() if w in q1words]
    total_weights = [weights.get(w, 0) for w in q1words] + [weights.get(w, 0) for w in q2words]
    
    R = np.sum(shared_weights) / np.sum(total_weights)
    return R


def dot_prod_word2Vec (row):
    question1 = str(row['question1'])
    v1 = convertToV(question1, glove50d_dict, 50)
    question2 = str(row['question2'])
    v2 = convertToV(question2, glove50d_dict, 50)
    return np.dot (v1, v2)
    
from collections import Counter

# If a word appears only once, we ignore it completely (likely a typo)
# Epsilon defines a smoothing constant, which makes the effect of extremely rare words smaller
def get_weight(count, eps=10000, min_count=2):
    if count < min_count:
        return 0
    else:
        return 1 / (count + eps)

train_qs = pd.Series(train['question1'].tolist() + train['question2'].tolist()).astype(str)
eps = 5000 
words = (" ".join(train_qs)).lower().split()
counts = Counter(words)
weights = {word: get_weight(count) for word, count in counts.items()}

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\minhq\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [29]:
#xgboost
X = pd.DataFrame()
X['word_match'] = train.apply(word_match_share, axis=1, raw=True)
X['tfidf_word_match'] = train.apply(tfidf_word_match_share, axis=1, raw=True)
X['dot_prod_word2Vec'] = train.apply (dot_prod_word2Vec, axis=1, raw = True)

y = train['is_duplicate'].values

C:\Users\minhq\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in double_scalars


In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [43]:
!pip install xgboost
import xgboost as xgb

# Set our parameters for xgboost
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.02
params['max_depth'] = 4

d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_val, label=y_val)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=50, verbose_eval=10)

  Using cached https://files.pythonhosted.org/packages/1d/e7/5258cb787dc036f419ec57491decf8bfa89ab52c401b08b4b9228e43dc4c/xgboost-0.81-py2.py3-none-win_amd64.whl


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 19.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


[16:36:15] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[0]	train-logloss:0.686228	valid-logloss:0.686272
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[16:36:15] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[16:36:16] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[16:36:16] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[16:36:16] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[16:36:16] C:\Users\Administrator\Desktop\xg

[16:36:24] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[16:36:24] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[16:36:24] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[16:36:24] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[16:36:25] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[16:36:25] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[16:36:25] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[16:36:33] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[16:36:33] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[16:36:33] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[16:36:33] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[16:36:33] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[16:36:33] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[110]	train-logloss:0.476291	valid-logloss:0.477503
[16:36:34] C:\Users\Administrator\Desktop\

[16:36:41] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[16:36:41] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[16:36:41] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[160]	train-logloss:0.46558	valid-logloss:0.466789
[16:36:41] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[16:36:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[16:36:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[16:36:42] C:\Users\Administrator\Desktop\x

[16:36:50] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[16:36:50] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[16:36:50] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[16:36:50] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[16:36:50] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[16:36:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[16:36:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

[16:37:00] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[16:37:00] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[16:37:00] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_depth=4
[16:37:00] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[16:37:00] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[16:37:01] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[270]	train-logloss:0.459199	valid-logloss:0.460638
[16:37:01] C:\Users\Administrator\Desktop\

[16:37:17] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[16:37:17] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_depth=4
[16:37:17] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_depth=4
[16:37:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[16:37:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[16:37:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[16:37:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning e

In [44]:
d_test = xgb.DMatrix(X_test)
pred = bst.predict(d_test)
array = []
for i in pred:
    if (i >= 0.5):
        array.append(1)
    else:
        array.append(0)
        
print(np.mean(array == y_test)) 
print(precision_recall(array, y_test))


0.7350540453634767
25951
33484
17595
[0.43662824934802724, 0.5959445184402701]


In [45]:
def isEqual(question1, question2):
    d = {'question1':[question1], 'question2': [question2]}    
    X = pd.DataFrame()
    X['word_match'] = [word_match_share(d)]
    X['tfidf_word_match'] = [tfidf_word_match_share(d)]
    X['dot_prod_word2Vec'] = [dot_prod_word2Vec (d)]
    d_test = xgb.DMatrix(X)
    pred = bst.predict(d_test)
    
    print(pred)
    for i in pred:
        if (i >= 0.5):
            print("These question are the same!!!")
        else:
            print ("SORRY, these questions are totally different")
            

In [49]:
isEqual("What is the meaning of life", "What is the meaning of death")

[0.20150375]
SORRY, these questions are totally different
